# 文件读写

无论什么编程语言，最后生成的程序，其运行过程大多如下所示：
1. 输入  
从键盘、硬盘、网络或其它设备获取数据
2. 处理  
运行各种功能、完成对数据进行处理任务
3. 输出  
把处理结果在屏幕显示、保存到硬盘，通过网络传送等。

对于输入输出来说，很多时候都会用硬盘数据永久存储起来。程序永久储存数据的方式有文件和数据库。数据文件是一个简单的方法。

实际上大多数程序都会涉及从硬盘文件中读取数据，最后再保存数据到文件中，这也是把文件读写单独列为一章。对于数据库这种数据持久化方式，可能要单独一本书才可以介绍的更好。

## 硬盘上的文件

在硬盘上的文件存储与管理，以及读写文件的功能是有操作系统提供的。

### 文件与目录

硬盘上的文件是以目录（directory）的形式进行组织管理的，也常常称为文件夹。路径是一个字符串，用来指明一个文件或目录。操作系统不同，路径的表示方法也有所不同。假定有一个包含学生成绩的文件，在不同系统下的路径会是：
- Windows系统，`E:\\data\\scores.csv`
- Linux系统，`\home\whwang\data\\scores.csv`

在Python标准库`os`中，提供了很多操作文件和目录的函数，详见标准库章节。

### 文件格式与编码

从本质上讲，硬盘存储的文件都是二进制的。根据编码逻辑，可以把文件分为文本和二进制文件。
- 文本文件是基于字符编码，其内容为人类可以阅读的文字。
- 二进制文件是基于值编码的文件，应用不同，遵循的编码也不同。

文本文件常见编码方法包括ASCII、UTF-8、Unicode等编码。参见字符串编码章节。

文本文件可读性强，使用记事本打开。常见的文本文件格式。


二进制文件遵循特定的编码格式，要借助特定工具才能为人认识。常见二进制文件如下所示：
- 可执行文件（exe,dll）
- 图像文件（bpm、jpg、gif
）
- 影像文件（mp3,mp4）
- office文件（doc, ppt, xls等）

## `open()`函数

磁盘文件的读写功能是由操作系统提供的，在Python中使用`open()`函数向操作系统请求打开一个文件对象（也称为文件描述符）；然后通过文件对象来读取数据，或者写入数据。

`open()`函数的语法为：
```
open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
```
其中主要参数为：
- `file`，位置参数，指定文件路径名
- `mode`，打开文件的模式，缺省是只读模式`rt`
- `encoding`，指定文件编码方式

In [32]:
help(open)

Help on built-in function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise IOError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position

### 打开模式

`open`文件的打开模式有如下选项：

| 模式  | 操作  | 说明  |
|:-----:|:-----:|:------|		
| `r` |	只读	| 默认模式，打开文件准备读取 |
| `w` |	只写	| 打开文件准备写入，会先清空文件内容|
| `x` |	新建	| 新建文件，并打开准备写入，比`w`模式更安全|
| `a` |	追加	| 打开文件准备写入，如果文件存在，从文件尾追加写入|
| `b` |	二进制模式	| 以二进制模式操作文件 |
| `t` |	文本文件模式	| 缺省值，以文本文件操作文件文件 |
| `+` |	读写模式	| 打开文件准备读写 |

### 打开文件

下面使用只读模式打开磁盘上的文件`assets/scores.csv`。

In [33]:
filename = 'assets/scores.csv' 
fh = open(filename)

`open()`函数会返回一个文件对象。如果文件不存在会引起`FileNotFoundError`的异常。

In [34]:
filename = 'unknownfile.txt'
fh = open(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'unknownfile.txt'

为此，在读写文件通常使用异常处理。

In [35]:
try:
    filename = 'unknownfile.txt'
    fh = open(filename)
except FileNotFoundError as e:
    print('{}'.format(e))    

[Errno 2] No such file or directory: 'unknownfile.txt'


文件打开成功，会进行文件的读取操作，最后需要调用文件对象的`close()`方法来关闭文件。可以使用`try...except...finally`语句来实现，最后都能正确地关闭文件。不过在Python中，可以使用`with`语句更简洁实现该功能。

In [36]:
try:
    filename = 'assets/scores.csv' 
    with open(filename) as fh:
        pass
except FileNotFoundError as e:
    print('{}'.format(e))        

## 文件对象

`open()`函数打开文件，会返回一个文件对象，使用该文件对象可以实现数据的读取和写入。

使用内置函数`type`查看对象：

In [2]:
filename = 'assets/scores.csv' 
fh = open(filename)

print(type(fh))

<class '_io.TextIOWrapper'>


### 属性和方法

#### 常用属性

| 方法  |  说明       |	
|:------|:---------------|
|`f.buffer`| 关闭文件。关闭后文件不能再进行读写操作。|
|`f.closed`| 刷新文件内部缓冲，直接把内部缓冲区的数据立刻写入文件, 而不是被动的等待输出缓冲区写入。。|
|`f.encoding`| 返回一个整型的文件描述符(file descriptor FD 整型), 可以用在如os模块的read方法等一些底层操作上。。|
|`f.line_buffering`| 如果文件连接到一个终端设备返回 True，否则返回 False。。|
|`f.name`| 返回文件下一行。。|
|`f.newlines`| 从文件读取指定的字节数，如果未给定或为负则读取所有。|

In [40]:
print(fh.buffer)
print(fh.closed)
print(fh.encoding)
print(fh.line_buffering)
print(fh.name)
print(fh.newlines)

<_io.BufferedReader name='assets/scores.csv'>
False
UTF-8
False
assets/scores.csv
None


#### 常用方法

| 方法  |  说明       |	
|:------|:---------------|
|`f.close()`| 关闭文件。关闭后文件不能再进行读写操作。|
|`f.flush()`| 刷新文件内部缓冲，直接把内部缓冲区的数据立刻写入文件, 而不是被动的等待输出缓冲区写入。。|
|`f.fileno()`| 返回一个整型的文件描述符(file descriptor FD 整型), 可以用在如os模块的read方法等一些底层操作上。。|
|`f.isatty()`| 如果文件连接到一个终端设备返回 True，否则返回 False。。|
|`f.next()`| 返回文件下一行。。|
|`f.read()`| 从文件读取指定的字节数，如果未给定或为负则读取所有。|
|`f.readline()`| 读取整行，包括 "\n" 字符。|
|`f.readlines()`| 读取所有行并返回列表，若给定sizeint>0，返回总和大约为sizeint字节的行, 实际读取值可能比 sizeint 较大, 因为需要填充缓冲区。|
|`f.seek()`| 设置文件当前位置|
|`f.tell()`| 返回文件当前位置。|
|`f.truncate([size])`| 从文件的首行首字符开始截断，截断文件为 size 个字符，无 size 表示从当前位置截断；截断之后后面的所有字符被删除，其中 Widnows 系统下的换行代表2个字符大小。|
|`f.write()`| 将字符串写入文件，返回的是写入的字符长度。|
|`f.writelines()`| 向文件写入一个序列字符串列表，如果需要换行则要自己加入每行的换行符。|

### 方法示例

** 读取文件 **

* `f.read()`  
读取字节

* `f.readline()`  
读取一行

* `f.readlines()`  
读取所有行

** 写入文件**

* `f.write()`
* `f.writelines()`

** 其他辅助函数 **

* `f.seek()`
* `f.tell()`
* `f.flush()`

In [41]:
filename = 'assets/scores.csv' 
fh = open(filename)
text = fh.read()
print(text)
fh.close()

Number   Name            Chinese    Math     Python
1        FengHaotong     80.0       90.0      62.0
2        CuiYuzhu        90.0       88.0      64.0
3        Shilingling     70.0       82.0      66.0
4        LiuZiang        60.0       76.0      68.0


In [42]:
filename = 'assets/scores.csv' 
fh = open(filename)
line = fh.readline()
print(line)
line = fh.readline()
print(line)
fh.close()

Number   Name            Chinese    Math     Python

1        FengHaotong     80.0       90.0      62.0



In [23]:
filename = 'assets/scores.csv' 
fh = open(filename)
lines = fh.readlines()
print(lines[0])
print(lines[-1])
fh.close()

Number   Name            Chinese    Math     Python

4        LiuZiang        60.0       76.0      68.0


## 示例-学生成绩单统计

学生成绩单保存在`scores.csv`文件里，每个学生一行，每行有学号，姓名，语文成绩，数学成绩，Python成绩，各栏目间用空格分隔。

要求统计每个学生的三科总分，并打印出来。

基本过程：
- 从scores.csv里读取学生成绩单；
- 统计三科总分（各科成绩均为浮点数）
- 打印结果

In [43]:
try:
    filename = 'assets/scores.csv' 
    with open(filename) as fh:
        # read the first line
        line = fh.readline()
        # read the data lines
        for line in fh.readlines():
            words = line.split()
            student_number = int(words[0])
            name = words[1]
            chinese = float(words[2])
            math = float(words[3])
            python = float(words[4])
            total = chinese + math + python
            print(student_number, name, total)
except FileNotFoundError as e:
    print('{}'.format(e))        

1 FengHaotong 232.0
2 CuiYuzhu 242.0
3 Shilingling 218.0
4 LiuZiang 204.0


## 应用

* 打开`scores.csv`，并读取文件；
* 对每一行进行解析，统计总分；
* 打开`scores.done.csv'`，并写入

In [44]:
%more assets/scores.csv

In [7]:
%%writefile sumscore.py
# Open the input and output file
inpfile = 'assets/scores.csv'
outfile = 'assets/scores.done.csv'

with open(inpfile, 'r') as inpfh, open(outfile, 'w') as outfh:
    # the first line
    headline = inpfh.readline()
    s = '{0}   total\n'.format(headline.strip())
    outfh.write(s)

    # for data lines
    s = '{0:>4}   {1:<16}  {2:4.1f}  {3:4.1f}  {4:4.1f}  {5:4.1f}\n'
    for line in inpfh:
        words = line.split()
        studentNumber = int(words[0])
        name = words[1]
        chinese = float(words[2])
        math = float(words[3])
        python = float(words[4])
        total = chinese + math + python
        outline = s.format(studentNumber, name, chinese, math, python, total)
        outfh.write(outline)

Overwriting sumscore.py


In [8]:
%run sumscore.py

In [9]:
%more assets/scores.done.csv

## 常用文件格式及其对应库

- 文本文件
- 二进制文件

### 文本文件格式

- CSV文件 （csv）
- INI文件 （configparser）
- yaml （pyyaml）
- JSON文件（json）
- HTML （lxml）

### 二进制文件格式

- excel （pandas, openpyxl）
- doc （python-docx）
- pdf （pdfminer）